In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, SimpleRNN
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud
import emoji
import string
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
%matplotlib inline

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

from nltk.corpus import stopwords
import pymorphy2

In [ ]:
from dateutil import parser as dtparser

In [ ]:
import pickle

In [ ]:
from datetime import datetime
import time

In [ ]:
from datetime import timedelta, date
import pyodbc

In [ ]:
import multiprocessing

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [ ]:
def sep_smiles(x):
    x=str(x)
    smile = re.findall(r'\){1,}', x)
    sad = re.findall(r'\({1,}', x)
    emojis = emoji.distinct_emoji_list(x)
    for word in smile:
        new_word=' '+'🙂'+' '
        x=x.replace(word, new_word)
    for word in sad:
        new_word=' '+'😞'+' '
        x=x.replace(word, new_word)
    for word in smile+sad+emojis:
        new_word=' '+word+' '
        x=x.replace(word, new_word)
    return(x)


In [ ]:
def clean(text):
  text=str(text)
  text = re.sub(r'[:;]', '', text)
  text=sep_smiles(text)
  xxx = re.findall(r'X{1,}', text)
  words = text.split(' ')
  new_words = []
  for word in words:
    if (word not in stopwords and word not in xxx):
      new_words.append(word)
  result=' '.join(new_words)
  result=" ".join(result.split())
  return result

In [ ]:
saved_model_path_neg = '/tmp/tf_save_neg'
saved_model_path_pos = '/tmp/tf_save_pos'
load_options = tensorflow.saved_model.LoadOptions(experimental_io_device='/job:localhost')
model_rnn_neg = tensorflow.keras.models.load_model(saved_model_path_neg, options=load_options)
model_rnn_pos = tensorflow.keras.models.load_model(saved_model_path_pos, options=load_options)

In [ ]:
def sent(text,neg_treshold,pos_treshold):
    text=clean(text)
    text=' '.join([morph.parse(word)[0]. normal_form for word in text.split()])
    sequence = tokenizer.texts_to_sequences([text])
    data = pad_sequences(sequence, maxlen=max_review_len)
    result_neg = model_rnn_neg.predict(data)
    result_pos = model_rnn_pos.predict(data)
    sent = new_label(result_neg,result_pos,neg_treshold,pos_treshold)
    return(sent,result_neg,result_pos)

In [ ]:
def new_label(pred_neg,pred_pos,neg_treshold,pos_treshold):
    if (pred_neg<neg_treshold and pred_pos<pos_treshold):
        label=0
    elif (pred_neg>=neg_treshold):
        label=-1
    elif (pred_pos>=pos_treshold):
        label=1
    else:
        label=0
    return(label)

In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
num_words = 10000
max_review_len = 100

In [ ]:
text='спасибо это было охуенно'
sent(text,0.6,0.85)[0]

1/1 [==============================] - 0s 127ms/step


0

In [ ]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [ ]:
start_date = date(2023, 4, 23)
end_date = date(2023, 4, 30)


for single_date in daterange(start_date, end_date):

    date=single_date.strftime("%Y-%m-%d")
    starttime = datetime.now()


    for h in range(24):

        server = ''
        database = ''
        username = ''
        password = ''
        driver= 'SQL Server'

        connection = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)



        requestString_sessions = f'''
        select top 500 TimeStart as Time,SessionID,MessageID,PhraseText
        from prd.PhraseData
        where cast(TimeStart as date)='{date}'
        and datepart(hour, TimeStart)={h}
        and NlpModelName!='AM'
        and len(PhraseText)>0
        '''

        #requestString_sessions = f'''
        #Select Time, SessionID, MessageID, PhraseText from ChatBotData.report.PhraseData_Just pdj
        #where cast(Time as date)='{date}' and datepart(hour, Time)={h} and PhraseText != ''
        #'''

        #requestString_sessions = f'''
        #Select top 500 Time, SessionID, MessageID, PhraseText from ChatBotData.report.PhraseData_Just pdj
        #where cast(Time as date)='{date}' and datepart(hour, Time)={h} and PhraseText != ''
        #and MessageID not in
            #(select MessageID from report.message_status_sentiment
             #where cast(Date as date)='{date}' and datepart(hour, Date)={h}) '''

        df = pd.read_sql_query(requestString_sessions,connection)

        starttime = datetime.now()
        df['Sentiment']=0

        with suppress_stdout():
            for i in range(len(df)):
                text=df['PhraseText'].iloc[i]
                s=sent(text,0.6,0.85)[0]
                df['Sentiment'].iloc[i]=s


        cursor = connection.cursor()

        for index, row in df.iterrows():
            cursor.execute("INSERT INTO [ChatBotData].[report].[message_status_sentiment] (Date,SessionID,MessageID,Sentiment) \
            values(?,?,?,?)", row.Time,row.SessionID,row.MessageID,row.Sentiment)

        print(date,h,len(df))

        connection.commit()
        cursor.close()

    print(datetime.now() - starttime)
    print(date)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2023-04-23 0 500
2023-04-23 1 500
2023-04-23 2 500
2023-04-23 3 500
2023-04-23 4 500
2023-04-23 5 500
2023-04-23 6 500
2023-04-23 7 500
2023-04-23 8 500
2023-04-23 9 500
2023-04-23 10 500
2023-04-23 11 500
2023-04-23 12 500
2023-04-23 13 500
2023-04-23 14 500
2023-04-23 15 500
2023-04-23 16 500
2023-04-23 17 500
2023-04-23 18 500
2023-04-23 19 500
2023-04-23 20 500
2023-04-23 21 500
2023-04-23 22 500
2023-04-23 23 500
0:00:51.023012
2023-04-23
2023-04-24 0 500
2023-04-24 1 500
2023-04-24 2 500
2023-04-24 3 500
2023-04-24 4 500
2023-04-24 5 500
2023-04-24 6 500
2023-04-24 7 500
2023-04-24 8 500
2023-04-24 9 500
2023-04-24 10 500
2023-04-24 11 500
2023-04-24 12 500
2023-04-24 13 500
2023-04-24 14 500
2023-04-24 15 500
2023-04-24 16 500
2023-04-24 17 500
2023-04-24 18 500
2023-04-24 19 500
2023-04-24 20 500
2023-04-24 21 500
2023-04-24 22 500
2023-04-24 23 500
0:00:51.133529
2023-04-24
2023-04-25 0 500
2023-04-25 1 500
2023-04-25 2 500
2023-04-25 3 500
2023-04-25 4 500
2023-04-25 5 500
20

Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]Общая ошибка сети. Обратитесь к документации по сети. (11)')